In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.metrics import root_mean_squared_log_error, root_mean_squared_error

In [9]:
oil_df = pd.read_csv('../data/oil.csv', parse_dates=['date'])
oil_df

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


In [26]:

dates = pd.date_range(oil_df['date'].min(), oil_df['date'].max())
dif1 = pd.Index(oil_df['date']).difference(dates)
dif2 = dates.difference(oil_df['date'])
dif2

DatetimeIndex(['2013-01-05', '2013-01-06', '2013-01-12', '2013-01-13',
               '2013-01-19', '2013-01-20', '2013-01-26', '2013-01-27',
               '2013-02-02', '2013-02-03',
               ...
               '2017-07-29', '2017-07-30', '2017-08-05', '2017-08-06',
               '2017-08-12', '2017-08-13', '2017-08-19', '2017-08-20',
               '2017-08-26', '2017-08-27'],
              dtype='datetime64[ns]', length=486, freq=None)

In [27]:
start = oil_df['date'].min()
end = oil_df['date'].max()
oil = oil_df.merge(
    pd.DataFrame({"date": pd.date_range(start, end)}),
    on="date",
    how="outer",
).sort_values("date", ignore_index=True)

In [28]:
oil.head(20)

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-05,NaN
5,2013-01-06,NaN
6,2013-01-07,93.20
7,2013-01-08,93.21
8,2013-01-09,93.08
9,2013-01-10,93.81


In [31]:
filled = oil['dcoilwtico'].interpolate('linear', limit_direction='both')
filled.head(20)

0     93.140000
1     93.140000
2     92.970000
3     93.120000
4     93.146667
5     93.173333
6     93.200000
7     93.210000
8     93.080000
9     93.810000
10    93.600000
11    93.823333
12    94.046667
13    94.270000
14    93.260000
15    94.280000
16    95.490000
17    95.610000
18    95.730000
19    95.850000
Name: dcoilwtico, dtype: float64

# Interpolate in group by

In [35]:
transaction = pd.read_csv('../data/transactions.csv', parse_dates=['date'])
sales = pd.read_csv('../data/train.csv', parse_dates=['date'])

In [36]:
transaction = transaction.merge(
    sales,
    on=["date", "store_nbr"],
    how="outer",
).sort_values(["date", "store_nbr"], ignore_index=True)

In [52]:
transaction.loc[transaction.sales.eq(0), "transactions"] = 0
transaction

,date,store_nbr,transactions,id,family,sales,onpromotion
0,2013-01-01,1,0.0,0,AUTOMOTIVE,0.000,0
1,2013-01-01,1,0.0,1,BABY CARE,0.000,0
2,2013-01-01,1,0.0,2,BEAUTY,0.000,0
3,2013-01-01,1,0.0,3,BEVERAGES,0.000,0
4,2013-01-01,1,0.0,4,BOOKS,0.000,0
...,...,...,...,...,...,...,...
3000883,2017-08-15,54,802.0,3000751,POULTRY,59.619,0
3000884,2017-08-15,54,802.0,3000752,PREPARED FOODS,94.000,0
3000885,2017-08-15,54,802.0,3000753,PRODUCE,915.371,76
3000886,2017-08-15,54,0.0,3000754,SCHOOL AND OFFICE SUPPLIES,0.000,0


In [51]:
transaction.groupby("store_nbr", group_keys=False)['transactions'].apply(
    lambda x: x
)

0            0.0
1            0.0
2            0.0
3            0.0
4            0.0
           ...  
3000883    802.0
3000884    802.0
3000885    802.0
3000886      0.0
3000887    802.0
Name: transactions, Length: 3000888, dtype: float64